In [1]:
from model import *
from utils import *
import tensorflow as tf
import numpy as np
import time
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
tf.app.flags.DEFINE_string('f', '', 'kernel')

flags = tf.app.flags

flags.DEFINE_string('ASC', 'qb', 'restaurant or laptop or qb')
flags.DEFINE_string('DSC', 'yelp', '{yelp, twitter} for restaurant & {amazon, twitter} for laptop')
flags.DEFINE_integer('batch_size', 128, 'number of example per batch')
flags.DEFINE_float('learning_rate', 0.001, 'learning rate')
#flags.DEFINE_integer('n_iter', 25, 'training iteration')
flags.DEFINE_integer('n_iter', 20, 'training iteration')
# We slightly modify the training procedure. Feeding all DSC data in ONE epoch can get better results.
flags.DEFINE_float('gamma', 0.1, '{0.1, 0.1, 0.9, 0.2} for {res+yelp, res+twitter, laptop+amazon, laptop+twitter')
flags.DEFINE_integer('embedding_dim', 300, 'dimension of word embedding')
flags.DEFINE_integer('position_dim', 100, 'dimension of position embedding')
flags.DEFINE_integer('max_sentence_len', 85, 'max number of tokens per sentence')
flags.DEFINE_integer('max_target_len', 25, 'max number of tokens per target')
flags.DEFINE_float('keep_prob1', 0.5, 'dropout keep prob1')
flags.DEFINE_float('keep_prob2', 1.0, 'dropout keep prob2')
# Parameters for capsule layers.
flags.DEFINE_integer('filter_size', 3, 'filter_size')
flags.DEFINE_integer('sc_num', 16, 'sc_num')
flags.DEFINE_integer('sc_dim', 16, 'sc_dim')
flags.DEFINE_integer('cc_num',  3, 'cc_num')
flags.DEFINE_integer('cc_dim', 24, 'cc_dim')
flags.DEFINE_integer('iter_routing', 3, 'routing iteration')
flags.DEFINE_bool("reuse_embedding", True, "reuse word embedding & id, True or False")
FLAGS = flags.FLAGS

In [3]:
def main(_):
    start_time = time.time()
    info = ''
    index = 0
    
    for name, value in FLAGS.__flags.items():
        value = value.value
        if index < 19:
            info += '{}:{}  '.format(name, value)
        if index in [5, 11]:
            info += '\n'
        index += 1
    print('\n{:-^80}'.format('Parameters'))
    print(info + '\n')
    
    print('---------')
    print(FLAGS.ASC)
    
    data_path = 'data/{}/'.format(FLAGS.ASC)
    
    if not FLAGS.reuse_embedding :
        print('Initialize Word Dictionary & Embedding')
        word_dict = data_init(data_path, FLAGS.DSC)
        w2v = init_word_embeddings(data_path, word_dict, FLAGS.DSC)
    else:
        print('Reuse Word Dictionary & Embedding')
        with open(data_path + FLAGS.DSC + '_word2id.txt', 'r', encoding='utf-8') as f:
            word_dict = eval(f.read())
        w2v = np.load(data_path + FLAGS.DSC + '_word_embedding.npy')
    
    #print(w2v)
    #print(word_dict)
    
    model = MODEL(FLAGS, w2v, word_dict, data_path)
    model.run()
    
    end_time = time.time()
    print('Running Time: {:.0f}m {:.0f}s'.format((end_time-start_time) // 60, (end_time-start_time) % 60))

In [4]:
if __name__ == '__main__':
    tf.app.run()


-----------------------------------Parameters-----------------------------------
logtostderr:False  alsologtostderr:False  log_dir:  v:0  verbosity:0  stderrthreshold:fatal  
showprefixforinfo:True  run_with_pdb:False  pdb_post_mortem:False  run_with_profiling:False  profile_file:None  use_cprofile_for_profiling:True  
only_check_args:False  op_conversion_fallback_to_while_loop:False  test_random_seed:301  test_srcdir:  test_tmpdir:/tmp/absl_testing  test_randomize_ordering_seed:  xml_output_file:  

---------
qb
Reuse Word Dictionary & Embedding



W0319 19:55:08.403247 140583602894656 module_wrapper.py:139] From /home/rmajumder/266proj/266finalproject/transcap-qb/model.py:46: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



W0319 19:55:08.418211 140583602894656 module_wrapper.py:139] From /home/rmajumder/266proj/266finalproject/transcap-qb/model.py:52: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



I am TransCap.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


W0319 19:55:08.450290 140583602894656 deprecation.py:506] From /home/rmajumder/266proj/266finalproject/transcap-qb/model.py:65: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


...

W0319 19:55:11.186606 140583602894656 module_wrapper.py:139] From /home/rmajumder/266proj/266finalproject/transcap-qb/model.py:124: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



W0319 19:55:11.203905 140583602894656 module_wrapper.py:139] From /home/rmajumder/266proj/266finalproject/transcap-qb/model.py:125: The name tf.global_variables_initializer is deprecated. Please use tf.compat.v1.global_variables_initializer instead.

/home/rmajumder/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



-------------------------------------Iter0--------------------------------------
train loss=14.568844, dev loss=4.489197, dev acc=0.5367, step=256
test acc=0.5611, test precision=0.1870, test recall=0.3333, test f1=0.2396
max step:0, early stop step:0

-------------------------------------Iter1--------------------------------------
train loss=13.782716, dev loss=3.948963, dev acc=0.5525, step=512
test acc=0.5716, test precision=0.2704, test recall=0.3862, test f1=0.3148
max step:0, early stop step:0

-------------------------------------Iter2--------------------------------------
train loss=12.304043, dev loss=3.431163, dev acc=0.5727, step=768
test acc=0.5928, test precision=0.6732, test recall=0.4930, test f1=0.3823
max step:0, early stop step:0

-------------------------------------Iter3--------------------------------------
train loss=11.556598, dev loss=3.325856, dev acc=0.5770, step=1024
test acc=0.6003, test precision=0.6012, test recall=0.5150, test f1=0.3956
max step:0, early

SystemExit: 

/home/rmajumder/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
